## 152: Verificación de Cuenta con Token JWT en FastAPI

Implementar un endpoint que verifique el token enviado en el correo de activación para validar la cuenta del usuario y activarla en la base de datos.

🧩 1. Endpoint /verificar-cuenta para validar token
Este endpoint recibe el token como parámetro de consulta, lo decodifica y activa la cuenta del usuario si el token es válido.

python
Copiar
Editar
from fastapi import APIRouter, HTTPException, Depends, Request
from sqlalchemy.orm import Session
import jwt

router = APIRouter()

@router.get("/verificar-cuenta")
def verificar_cuenta(token: str, db: Session = Depends(get_db)):
    try:
        datos = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
        email = datos.get("sub")
    except jwt.ExpiredSignatureError:
        raise HTTPException(status_code=400, detail="Token expirado")
    except jwt.PyJWTError:
        raise HTTPException(status_code=400, detail="Token inválido")

    usuario = db.query(User).filter_by(email=email).first()
    if not usuario:
        raise HTTPException(status_code=404, detail="Usuario no encontrado")

    if usuario.is_active:
        return {"mensaje": "✅ Cuenta ya verificada"}

    usuario.is_active = True
    db.commit()

    return {"mensaje": "🎉 Cuenta verificada correctamente"}
🧩 2. Recordatorio del Token
El token fue generado con JWT y enviado al usuario en el correo, con una validez de 24 horas:

python
Copiar
Editar
expira = datetime.utcnow() + timedelta(hours=24)
datos = {"sub": usuario.email, "exp": expira}
token = jwt.encode(datos, SECRET_KEY, algorithm=ALGORITHM)